<a href="https://colab.research.google.com/github/ganiosh92/DA-ecommerce-clickstream-analytics/blob/main/clickstream_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pwd
%cd /content/drive/My Drive/Sample-Data-From-Web/ecommerce_customer_behavior

/content
/content/drive/My Drive/Sample-Data-From-Web/ecommerce_customer_behavior


In [3]:
import pandas as pd
import numpy as np
import json
from pandas import json_normalize

In [4]:
#customer = pd.read_csv('customer.csv')
clickstream = pd.read_csv('click_stream.csv')
#transactions = pd.read_csv('transactions.csv')
#product = pd.read_csv('product.csv', on_bad_lines='skip')

In [ ]:
clickstream.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12833602 entries, 0 to 12833601
Data columns (total 6 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   session_id      object
 1   event_name      object
 2   event_time      object
 3   event_id        object
 4   traffic_source  object
 5   event_metadata  object
dtypes: object(6)
memory usage: 587.5+ MB


In [ ]:
clickstream.head()

,session_id,event_name,event_time,event_id,traffic_source,event_metadata
0,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c,HOMEPAGE,2019-09-06T15:54:32.821085Z,9c4388c4-c95b-4678-b5ca-e9cbc0734109,MOBILE,NaN
1,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c,SCROLL,2019-09-06T16:03:57.821085Z,4690e1f5-3f99-42d3-84a5-22c4c4d8500a,MOBILE,NaN
2,7d440441-e67a-4d36-b324-80ffd636d166,HOMEPAGE,2019-09-01T12:05:10.322763Z,88aeaeb5-ec98-4859-852c-8abb483faf31,MOBILE,NaN
3,7d440441-e67a-4d36-b324-80ffd636d166,ADD_TO_CART,2019-09-01T12:06:33.322763Z,934e306e-ecc6-472f-9ccb-12c8536910a2,MOBILE,"{'product_id': 15315, 'quantity': 4, 'item_pri..."
4,7d440441-e67a-4d36-b324-80ffd636d166,BOOKING,2019-09-01T12:15:29.425431Z,9f4767a1-40fa-4c9c-9524-dfad18634d56,MOBILE,{'payment_status': 'Success'}


In [ ]:
#create a function to convert event_metadata column to JSON format
def func(a):
  return(json.loads(a.replace("'",'"')))

In [ ]:
#flatten the event_metadata column

#drop rows with na values
clickstream_no_na = clickstream.dropna().copy()

#create a new column and fill it with dummy value = 1
clickstream_no_na['test_col'] = "1"

#apply the function to convert event_metadata column values from string to JSON
clickstream_no_na['test_col'] = clickstream_no_na['event_metadata'].apply(func)

#impute the flattened columns to the old dataset
clickstream_no_na_flattened = pd.concat([clickstream_no_na, json_normalize(clickstream_no_na['test_col'])], axis=1)

In [ ]:
clickstream_no_na_flattened.head()

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,test_col,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount
3,7d440441-e67a-4d36-b324-80ffd636d166,ADD_TO_CART,2019-09-01T12:06:33.322763Z,934e306e-ecc6-472f-9ccb-12c8536910a2,MOBILE,"{'product_id': 15315, 'quantity': 4, 'item_pri...","{'product_id': 15315, 'quantity': 4, 'item_pri...",6133.0,2.0,249443.0,NaN,NaN,NaN,NaN
4,7d440441-e67a-4d36-b324-80ffd636d166,BOOKING,2019-09-01T12:15:29.425431Z,9f4767a1-40fa-4c9c-9524-dfad18634d56,MOBILE,{'payment_status': 'Success'},{'payment_status': 'Success'},6721.0,1.0,134504.0,NaN,NaN,NaN,NaN
5,7d440441-e67a-4d36-b324-80ffd636d166,SEARCH,2019-09-01T12:07:00.322763Z,c952142b-4fe9-4694-ad7f-21a5d1bed9ca,MOBILE,{'search_keywords': 'Dress Kondangan'},{'search_keywords': 'Dress Kondangan'},57622.0,1.0,384716.0,NaN,NaN,NaN,NaN
12,f7b5cd60-4f87-4b3c-8bde-c7da8930eec2,ADD_TO_CART,2019-09-09T05:06:03.425431Z,c1b42157-39a7-4521-a4dc-ed0200de301a,MOBILE,"{'product_id': 6133, 'quantity': 2, 'item_pric...","{'product_id': 6133, 'quantity': 2, 'item_pric...",NaN,NaN,NaN,Success,NaN,NaN,NaN
13,f7b5cd60-4f87-4b3c-8bde-c7da8930eec2,ADD_TO_CART,2019-09-09T05:23:36.425431Z,61cbe834-c585-406e-8eac-30f671e51fbb,MOBILE,"{'product_id': 6721, 'quantity': 1, 'item_pric...","{'product_id': 6721, 'quantity': 1, 'item_pric...",NaN,NaN,NaN,NaN,Tas Wanita,NaN,NaN


## TESTING

In [ ]:
#Test
test = clickstream_no_na_flattened.copy()

In [ ]:
#get the dataset where event_name is not a na/nan value
test2 = test[test["event_name"].isna() == False]

In [ ]:
test2.head(1)

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,test_col,product_id,quantity,item_price,payment_status,search_keywords,promo_code,promo_amount
3,7d440441-e67a-4d36-b324-80ffd636d166,ADD_TO_CART,2019-09-01T12:06:33.322763Z,934e306e-ecc6-472f-9ccb-12c8536910a2,MOBILE,"{'product_id': 15315, 'quantity': 4, 'item_pri...","{'product_id': 15315, 'quantity': 4, 'item_pri...",6133.0,2.0,249443.0,NaN,NaN,NaN,NaN


In [ ]:
#find distinct event_name types
set(test2['event_name'])

{'ADD_PROMO', 'ADD_TO_CART', 'BOOKING', 'SEARCH'}

In [ ]:
#compute the number of "ADD_TO_CART" events that led to a buy
add_to_cart_num = len(test2[test2["event_name"] == 'ADD_TO_CART'])
add_to_cart_buy_num = len(test2[(test2['event_name'] == 'ADD_TO_CART') & (test2['payment_status'] == 'Success')])

#percentage to ADD_TO_CART's that resulted in a buy
atc_pct = round((add_to_cart_buy_num / add_to_cart_num) * 100, 2)

print(str(atc_pct) + "%")

5.06%


## Traffic Source

In [ ]:
mobile_traffic = round((len(clickstream[clickstream['traffic_source'] == 'MOBILE']) / len(clickstream)) * 100, 2)
web_traffic = round((len(clickstream[clickstream['traffic_source'] == 'WEB']) / len(clickstream)) * 100, 2)

print("Percentage of Mobile Traffic = " + str(mobile_traffic)+"%")
print("Percentage of Web Traffic = " + str(web_traffic)+"%")

Percentage of Mobile Traffic = 89.98%
Percentage of Web Traffic = 10.02%


#Different Event Types and their Description

In [ ]:
print(set(clickstream['event_name']))

#All Events
#1. HOMEPAGE = ------------- --> all N/A values
#2. CLICK = ------------- --> all N/A values
#3. ITEM_DETAIL = ------------- --> all N/A values
#4. SCROLL = ------------- --> all N/A values
#5. PROMO_PAGE = ------------- --> all N/A values
#6. BOOKING = payment, can result in either 'payment failed' or 'payment succeeded' --> no N/A values
#7. ADD_PROMO = Adding the Promo code and then seeing a promo amount BUT DID NOT RESULT IN A BUY--> no N/A values
#8. ADD_TO_CART = Act of adding certain products to cart BUT DID NOT RESULT IN A BUY. Has product_id, quantity and item_price --> no N/A values
#9. SEARCH = list of search keywords entered by the user --> No N/A values

print(" --- ")

print("% of Each Event")
print("1. Homepage: " + str(round(len(clickstream[clickstream['event_name'] == 'HOMEPAGE'])/len(clickstream),2)*100) + "%")
print("2. Click: " + str(round(len(clickstream[clickstream['event_name'] == 'CLICK'])/len(clickstream),2)*100)+ "%")
print("3. Item Detail: " + str(round(len(clickstream[clickstream['event_name'] == 'ITEM_DETAIL'])/len(clickstream),2)*100)+ "%")
print("4. Scroll: " + str(round(len(clickstream[clickstream['event_name'] == 'SCROLL'])/len(clickstream),2)*100)+ "%")
print("6. Booking: " + str(round(len(clickstream[clickstream['event_name'] == 'BOOKING'])/len(clickstream),2)*100)+ "%")
print("7. Add Promo: " + str(round(len(clickstream[clickstream['event_name'] == 'ADD_PROMO'])/len(clickstream),2)*100)+ "%")
print("8. Add to Cart: " + str(round(len(clickstream[clickstream['event_name'] == 'ADD_TO_CART'])/len(clickstream),2)*100)+ "%")
print("9. Search: " + str(round(len(clickstream[clickstream['event_name'] == 'SEARCH'])/len(clickstream),2)*100)+ "%")



{'SEARCH', 'BOOKING', 'ITEM_DETAIL', 'PROMO_PAGE', 'ADD_TO_CART', 'ADD_PROMO', 'CLICK', 'HOMEPAGE', 'SCROLL'}
 --- 
% of Each Event
1. Homepage: 19.0%
2. Click: 19.0%
3. Item Detail: 10.0%
4. Scroll: 13.0%
6. Booking: 7.000000000000001%
7. Add Promo: 3.0%
8. Add to Cart: 15.0%
9. Search: 9.0%


# Preprocessing ***Clickstream*** Data

In [22]:
#PREPROCESSING
#extract the mm/dd/yyyy from the date field
clickstream['event_time'] = pd.to_datetime(clickstream['event_time'])
clickstream['event_day'] = clickstream['event_time'].dt.day
clickstream['event_month'] = clickstream['event_time'].dt.month
clickstream['event_year'] = clickstream['event_time'].dt.year
clickstream['unique_session_id'] = clickstream['session_id'] + clickstream['event_time'].dt.strftime('%Y-%m-%d')

clickstream.head(5)

,session_id,event_name,event_time,event_id,traffic_source,event_metadata,event_day,event_month,event_year,unique_session_id
0,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c,HOMEPAGE,2019-09-06 15:54:32.821085+00:00,9c4388c4-c95b-4678-b5ca-e9cbc0734109,MOBILE,NaN,6,9,2019,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c2019-09-06
1,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c,SCROLL,2019-09-06 16:03:57.821085+00:00,4690e1f5-3f99-42d3-84a5-22c4c4d8500a,MOBILE,NaN,6,9,2019,fb0abf9e-fd1a-44dd-b5c0-2834d5a4b81c2019-09-06
2,7d440441-e67a-4d36-b324-80ffd636d166,HOMEPAGE,2019-09-01 12:05:10.322763+00:00,88aeaeb5-ec98-4859-852c-8abb483faf31,MOBILE,NaN,1,9,2019,7d440441-e67a-4d36-b324-80ffd636d1662019-09-01
3,7d440441-e67a-4d36-b324-80ffd636d166,ADD_TO_CART,2019-09-01 12:06:33.322763+00:00,934e306e-ecc6-472f-9ccb-12c8536910a2,MOBILE,"{'product_id': 15315, 'quantity': 4, 'item_pri...",1,9,2019,7d440441-e67a-4d36-b324-80ffd636d1662019-09-01
4,7d440441-e67a-4d36-b324-80ffd636d166,BOOKING,2019-09-01 12:15:29.425431+00:00,9f4767a1-40fa-4c9c-9524-dfad18634d56,MOBILE,{'payment_status': 'Success'},1,9,2019,7d440441-e67a-4d36-b324-80ffd636d1662019-09-01


#Bounce Rate
the percentage of visitors who arrive at a digital storefront and “bounce,” leaving before ever progressing onto a second page. Bounce rates are calculated by dividing the number of one-page visits by the total number of visits to the site.

In [67]:
#Bounce rate in 2019
#Here we define bounce rate as when user arrives at the HOMPAGE or PROMO_PAGE but doesn't perform the following actions:
#1. CLICK i.e clicks on elements on the webpage
#2. SCROLL i.e. scrolls the webpage
#3. BOOKING i.e. pays or makes a transaction
#4. SEARCH i.e. searches for an item
#5. ADD_TO_CART i.e adds items to cart
#6. ITEM_DETAIL i.e. views an item
#7. ADD_PROMO i.e. adds promo coupon etc
for i in range(1,12):

  events = clickstream[(clickstream['event_year']==2019) & (clickstream['event_month'] == i)]
  events = events.sort_values(by = 'event_time', ascending = True)
  unique_sessions = pd.DataFrame(events.groupby('unique_session_id')['event_name'].agg(list))

  unique_sessions_with_one_event = unique_sessions[unique_sessions['event_name'].apply(lambda x: len(x) == 1)]

  unique_bounced_sessions = unique_sessions_with_one_event[unique_sessions_with_one_event['event_name'].apply(lambda x: x[0]).isin(['HOMEPAGE', 'PROMO_PAGE'])]



  print(str(i) + ": " + str(len(unique_sessions)) + ", " + str(len(unique_bounced_sessions)))

1: 45198, 7847
2: 41813, 7405
3: 48709, 8575
4: 47868, 8315
5: 52213, 9042
6: 51311, 8660
7: 56323, 9649
8: 58477, 10191
9: 58375, 10173
10: 61993, 10603
11: 62677, 10936


In [63]:
unique_sessions_with_one_event[unique_sessions_with_one_event['event_name'].apply(lambda x: x[0]).isin(['HOMEPAGE', 'PROMO_PAGE'])]

,event_name
unique_session_id,
0007a00f-430a-4530-830f-47c3c9b77ecd2019-11-16,[HOMEPAGE]
00119d08-334f-4538-8443-1e7488605df72019-11-07,[HOMEPAGE]
00119d08-334f-4538-8443-1e7488605df72019-11-12,[HOMEPAGE]
00184cd6-c7c8-499d-8cea-ed301d87bcf32019-11-01,[HOMEPAGE]
00195c45-aac7-40c9-bdd4-2cc3c4afb2d62019-11-09,[HOMEPAGE]
...,...
ffe10a3c-21c0-4d94-9e13-94ef21c6e4e32019-11-09,[HOMEPAGE]
ffe10a3c-21c0-4d94-9e13-94ef21c6e4e32019-11-13,[HOMEPAGE]
ffe948be-c9c9-4c06-9baa-bc16b7df211f2019-11-17,[PROMO_PAGE]


In [64]:
len(unique_sessions_with_one_event)

34784